<a href="https://colab.research.google.com/github/vegger/MLDM/blob/main/labs/L05_Supervised_Learning_LAB_ASSIGNMENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
RANDOM_SEED = 0x0

# TASK 1. Metrics (4 Points): 
In this task you will compute some standard quality measures like Precision, Recall and F-Score for an artificial dataset.

First, we generate some artificial data for a classification task and take a look at it.

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X, y = make_blobs(n_samples=500, centers=4, random_state=RANDOM_SEED, cluster_std=2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
for target_class in np.unique(y):
  plt.scatter(X_train[y_train==target_class, 0], X_train[y_train==target_class, 1], alpha=0.75, label=target_class)
plt.legend()
plt.show()

As we can see, the classes are not easy distinguishable. That is not an easy task for the model. 
Nevertheless, let's apply a Logistic Regression Model and predict `y` values. 

In [ ]:
log_reg = LogisticRegression(random_state=RANDOM_SEED).fit(X_train, y_train)
y_test_pred = log_reg.predict(X_test)

### Task 1a. Create and visualise the confusion matrix that describes the results:

1. Create confusion matrix. Use can use `sklearn.metrics.confusion_matrix` functions.
2. Display the confusion matrix of the Logistic Regression Model with `seaborn.heatmap`. Include the numbers of samples in each cell of the heatmap.

### Task 1b. Create functions that calculate Precision, Recall and F1-Score. 
1. Implement your own functions for calculating Precision, Recall and F1-Score. Don't use any of the existing libraries for this.
2. Apply them on the example above for the class `0`.

In [ ]:
def precision(class_of_interest, confusion_matrix_test):
  ...
  return score

def recall(class_of_interest, confusion_matrix_test):
  ...
  return score

def f1_score(precision_value, recall_value):
  ...
  return score

In [ ]:
precision_value = precision(...)
recall_value = recall(...)
f1_score_test = f1_score(...)

print(f"Precision: {precision_value}")
print(f"Recall: {recall_value}")
print(f"F1-Score: {f1_score_test}")

### Task 1c. Check your results 
Use the function `classification_report` of `sklearn.metrics` to compare their results to your own implementation

In [ ]:
from sklearn.metrics import classification_report

print(...)

## 📢 **HAND-IN** 📢: Post following results you got in this task:

1. Confusion matrix from task 1a
2. Precision, Recall and F1-Score from task 1b



# TASK 2. Cross Validation (1 point)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import randrange
import seaborn as sns
sns.set() # just the theme

In this task, we familiarize ourselves with how different types of Cross Validation actually partition the data. 

Run the following code and inspect the graphs that show how the data was split for different runs. Full code source can be found [here](https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html)

Note: the bar "class" on the graphs shows the samples of 3 classes. First, all samples of class 0 are listed (in light blue color), then all samples of class 1 are shown (yellow) and, lastly, all samples of class 2 are following (brown).

In [ ]:
from sklearn.model_selection import (
    KFold,
    ShuffleSplit,
    StratifiedKFold,
    StratifiedShuffleSplit
)
from matplotlib.patches import Patch

# first, generate dummy data
n_points = 100
rng = np.random.RandomState(RANDOM_SEED)
X = rng.randn(100, 10)

# we have 10% of the data belonging to class 0, 30% - to class 1 and 60% to class 2
# first come all the data samples that belong to class 0, following by the samples of class 1 and then of class 2
percentiles_classes = [0.1, 0.3, 0.6]
y = np.hstack([[ind] * int(100 * perc) for ind, perc in enumerate(percentiles_classes)]) 

# color settings for graphs
cmap_data = plt.cm.Paired
cmap_cv = plt.cm.coolwarm

n_splits = 5

In [ ]:
cvs = [KFold, ShuffleSplit, StratifiedKFold, StratifiedShuffleSplit]

def plot_cv_indices(cv, X, y, ax, n_splits, lw=10):
    """Create a sample plot for indices of a cross-validation object."""

    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv.split(X=X, y=y)):
      
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1.0
        indices[tr] = 0

        # Visualize the results
        ax.scatter(
            range(len(indices)),
            [ii + 0.5] * len(indices),
            c=indices,
            marker="_",
            lw=lw,
            cmap=cmap_cv,
            vmin=-0.2,
            vmax=1.2
        )

    # Plot the data classes and groups at the end
    ax.scatter(
        range(len(X)), [ii + 1.5] * len(X), c=y, marker="_", lw=lw, cmap=cmap_data
    )
    # Formatting
    yticklabels = list(range(n_splits)) + ["class"]
    ax.set(
        yticks=np.arange(n_splits + 1) + 0.5,
        yticklabels=yticklabels,
        xlabel="Sample index",
        ylabel="CV iteration",
        ylim=[n_splits + 1.2, -0.2],
        xlim=[0, 100],
    )
    ax.set_title("{}".format(type(cv).__name__), fontsize=15)
    return ax

for cv in cvs:
    fig, ax = plt.subplots(figsize=(6, 3))
    plot_cv_indices(cv(n_splits), X, y, ax, n_splits)



## 📢 **HAND-IN** 📢: Answer following question in Moodle:

Try to explain in your own words (1-2 lines) how ShuffleSplit splits the data. 



# Task 3. Cross Validation and Imbalanced Data (5 points)
In this task we are working with the **Default of Credit Card Clients Dataset**. This dataset contains information on default payments, demographic factors, credit data, history of payment, and bill statements of credit card clients in Taiwan from April 2005 to September 2005. More info about the dataset can be found [here](https://www.kaggle.com/datasets/uciml/default-of-credit-card-clients-dataset).

The target variable is:

- `default.payment.next.month`: Default payment (1=yes, 0=no)

Default payment means a missed payment. So, the target variable shows whether a person will miss his or her Credit Card Payment (=1) or will pay it back (=0).

The predictor features are:
*   ID: ID of each client
*   LIMIT_BAL: Amount of given credit in NT dollars (includes individual and family/supplementary credit
*   SEX:0, SEX:1: Binary variables for gender (0=male, 1=female)
*   EDUCATION:0-EDUCATION:5: Binary variables for education (0=graduate school, 1=university, 2=high school, 3=others, 4=unknown, 5=unknown)
*   MARRIAGE:0-MARRIAGE:2: Binary variables for Marital status (0=married, 1=single, 2=others)
*   AGE: Age in years
*   PAY_0: Repayment status in September, 2005 (-1=pay duly, 1=payment delay for one month, 2=payment delay for two months, … 8=payment delay for eight months, 9=payment delay for nine months and above)
*   PAY_2: Repayment status in August, 2005 (scale same as above)
*   PAY_3: Repayment status in July, 2005 (scale same as above)
*   PAY_4: Repayment status in June, 2005 (scale same as above)
*   PAY_5: Repayment status in May, 2005 (scale same as above)
*   PAY_6: Repayment status in April, 2005 (scale same as above)
*   BILL_AMT1: Amount of bill statement in September, 2005 (NT dollar)
*   BILL_AMT2: Amount of bill statement in August, 2005 (NT dollar)
*   BILL_AMT3: Amount of bill statement in July, 2005 (NT dollar)
*   BILL_AMT4: Amount of bill statement in June, 2005 (NT dollar)
*   BILL_AMT5: Amount of bill statement in May, 2005 (NT dollar)
*   BILL_AMT6: Amount of bill statement in April, 2005 (NT dollar)
*   PAY_AMT1: Amount of previous payment in September, 2005 (NT dollar)
*   PAY_AMT2: Amount of previous payment in August, 2005 (NT dollar)
*   PAY_AMT3: Amount of previous payment in July, 2005 (NT dollar)
*   PAY_AMT4: Amount of previous payment in June, 2005 (NT dollar)
*   PAY_AMT5: Amount of previous payment in May, 2005 (NT dollar)
*   PAY_AMT6: Amount of previous payment in April, 2005 (NT dollar)


In [ ]:
# install datasets
!pip install datasets

First, we load and inspect the data.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("imodels/credit-card")
df_train = pd.DataFrame(dataset['train'])
X_train = df_train.drop(columns=['default.payment.next.month'])
y_train = df_train['default.payment.next.month'].values
print(X_train.head())

df_test = pd.DataFrame(dataset['test'])
X_test = df_test.drop(columns=['default.payment.next.month'])
y_test = df_test['default.payment.next.month'].values

In the following, we want to explore how to handle highly imbalanced data. "Unfortunately", the dataset is currently not very imbalenced. For this reason we implement an auxiliary function that deletes X% of all samples of class 1. Applying this function several times will make our data more and more imbalanced

In [ ]:
def delete_random_samples_of_class_1(samples_X, samples_y, percentage=90):
  # find samples of class 1
  y1_indices = np.argwhere(samples_y==1)
  y1_indices = y1_indices.reshape((y1_indices.shape[0],))
  sampled_indices = np.random.choice(y1_indices, int(percentage/100*len(y1_indices)), replace=False)
  # delete randomly chosen samples of class 1
  new_samples_X = np.delete(samples_X.values, sampled_indices, axis=0)
  new_samples_y = np.delete(samples_y, sampled_indices)
  return new_samples_X, new_samples_y

### 3a. Inspect target variable and apply function that increases the imbalance of the dataset
1. Check how many samples of each class are present in the train and test set
2. Apply `delete_random_samples_of_class_1`
3. Check again how many samples of each class are present in the train and test set

### 3b. Standardization the data
Apply Standardization to the new data. You could either use the functions you created for the previous labs or `StandardScaler` from the `sklearn` library.

### 3c. Logistic Regression and Cross Validation
Wer now apply Logistic Regression and one of the Cross Validation techniques from the example above to the data in order to predict `default.payment.next.month`. Proceed as follows:
1. Set `n_splits` to the number of splits that results in 20% of validation data in each split, set maximum iterations to `500` and random state to `RANDOM_SEED`.
2. Create and train the model
3. Apply the model in order to predict target variable of the test set. 
4. Calculate Accuracy, Precision (macro) and Recall (macro) for the predicted values.
5. Generate and visualize the confusion matrix of the test set.


In [ ]:
from sklearn.linear_model import LogisticRegressionCV

def logistic_regression_CV(n_splits, class_weight=None):
  ...
  return y_pred

In [ ]:
def return_statistics(y_true, y_pred):
  accuracy = ...
  precision = ...
  recall = ...

  print(f"Accuracy: {accuracy}")
  print(f"Precision (macro): {precision}")
  print(f"Recall (macro): {recall}")

  #visualise confusion matrix

In [ ]:
y_pred = logistic_regression_CV(...)
return_statistics(y_test, y_pred)

### 3d. Handling imbalanced dataset
1. Run the function `logistic_regression_CV` one more time. But this time add parameter `class_weight='balanced'` to the Logistic Regression Model. Adjust the function accordingly. 
2. Visualize the confusion matrix of the test set again.


In [ ]:
y_pred = ...  # train logistic regression with class_weight='balanced', use cross-validation as in the 3c
return_statistics(y_test, y_pred)

## 📢 **HAND-IN** 📢: Please hand in the following:

- The confusion matrices of 3c and 3d
- How do you interpret the confusion matrix of 3c?
- What is the major change that happend in 3d?


